In [1]:
import numpy as np
import pandas as pd

from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

## Examine data

In [2]:
%%capture
df = pd.read_csv('clinvar_20180225.ann.tsv.gz', compression='gzip', sep='\t', error_bad_lines=False, header=0)

In [3]:
len(df)

372159

In [4]:
df.head()

,#id,chrom,inputPos,inputRef,inputAlt,unannotatedReason,gene,geneId,geneDesc,transcript,...,granthamDist,AGVGDclass,AGVGDgv,AGVGDgd,SIFTprediction,SIFTweight,SIFTmedian,MAPPprediction,MAPPpValue,MAPPpValueMedian
0,475283,1,949422,G,A,-,ISG15,4053,ISG15 ubiquitin-like modifier,NM_005101.3,...,46.0,C0,178.468,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,183381,1,949523,C,T,-,ISG15,4053,ISG15 ubiquitin-like modifier,NM_005101.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,475278,1,949597,C,T,-,ISG15,4053,ISG15 ubiquitin-like modifier,NM_005101.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,402986,1,949608,G,A,-,ISG15,4053,ISG15 ubiquitin-like modifier,NM_005101.3,...,46.0,C0,130.977,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,161455,1,949696,C,CG,-,ISG15,4053,ISG15 ubiquitin-like modifier,NM_005101.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.dtypes

#id                       int64
chrom                    object
inputPos                  int64
inputRef                 object
inputAlt                 object
unannotatedReason        object
gene                     object
geneId                    int64
geneDesc                 object
transcript               object
strand                    int64
transLen                  int64
cdsLen                    int64
protein                  object
Uniprot                  object
varType                  object
codingEffect             object
varLocation              object
assembly                 object
gDNAstart                 int64
gDNAend                   int64
gNomen                   object
cDNAstart                object
cDNAend                  object
cNomen                   object
pNomen                   object
alt_pNomen               object
exon                      int64
intron                  float64
omimId                  float64
                         ...   
wtAA_3  

In [6]:
df.isnull().sum()

#id                          0
chrom                        0
inputPos                     0
inputRef                     0
inputAlt                     0
unannotatedReason            0
gene                         0
geneId                       0
geneDesc                     0
transcript                   0
strand                       0
transLen                     0
cdsLen                       0
protein                  16608
Uniprot                  88433
varType                      0
codingEffect            131716
varLocation                  0
assembly                     0
gDNAstart                    0
gDNAend                      0
gNomen                       0
cDNAstart                    0
cDNAend                      0
cNomen                       0
pNomen                    2380
alt_pNomen                2380
exon                         0
intron                  318115
omimId                   21182
                         ...  
wtAA_3                  158696
wtCodon 

## Preprocessing

In [7]:
RELEVANT_FEATURES = [
    'chrom',
    'inputPos',
    'inputRef',
    'inputAlt',
    'transcript',
    'codingEffect',
    'varLocation',
    'alt_pNomen',
    'wtSSFScore',
    'wtMaxEntScore',
    'varSSFScore',
    'varMaxEntScore',
    'rsId',
    'rsClinicalSignificance',
    'rsMAF',
    '1000g_AF',
    'gnomadAltFreq_all',
    'espAllMAF',
    'espAllAAF',
    'clinVarMethods',
    'clinVarClinSignifs',
    'nOrthos',
    'conservedOrthos'
]


RULES = [
    lambda df: df['codingEffect'] != 'synonymous',
    lambda df: df['varLocation'] != 'intron',
    lambda df: np.invert(df['1000g_AF'] > 0.01),
    lambda df: np.invert(df['gnomadAltFreq_all'] > 0.01)
]


def rules_filter(df):
    for rule in RULES:
        df = df[rule(df)]
    return df


df = df[RELEVANT_FEATURES]  # Filtering relevant features
df = rules_filter(df)  # Rules filtering

In [8]:
np.random.seed(42)
df = df.sample(frac=1).reset_index(drop=True)

## Making target

In [9]:
def pick_target(df, column='clinVarClinSignifs'):
    df = df.copy()
    df['y'] = df[column].astype(str).apply(lambda x: int('pathogenic' in x.lower()))
    del df[column]
    return df

In [10]:
df = pick_target(df)
df.head()

,chrom,inputPos,inputRef,inputAlt,transcript,codingEffect,varLocation,alt_pNomen,wtSSFScore,wtMaxEntScore,...,rsClinicalSignificance,rsMAF,1000g_AF,gnomadAltFreq_all,espAllMAF,espAllAAF,clinVarMethods,nOrthos,conservedOrthos,y
0,9,98231113,C,T,NM_000264.3,missense,exon,p.Glu724Lys,81.5220,7.45706,...,NaN,0.0002,0.0002,0.000022,0.000077,0.000077,clinical testing,11.0,6.0,0
1,X,25023006,G,GGGGGCCATTGTGGAAAAGAGCCTGCAGGGAGAGCAAACAGCGCGG...,NM_139058.2,frameshift,exon,p.Leu491Alafs*19,89.7583,10.89940,...,NaN,NaN,NaN,NaN,NaN,NaN,clinical testing,NaN,NaN,1
2,3,52486140,C,T,NM_003280.2,missense,exon,p.Asp62Asn,80.6382,9.43568,...,NaN,NaN,NaN,NaN,NaN,NaN,clinical testing,19.0,15.0,0
3,11,72004597,G,GC,NM_030813.5,frameshift,exon,p.Cys647Leufs*26,78.7932,8.95460,...,NaN,NaN,NaN,NaN,NaN,NaN,literature only,NaN,NaN,1
4,12,6105290,C,A,NM_000552.4,stop gain,exon,p.Glu1981*,93.3547,9.90740,...,not_provided,0.0000,NaN,NaN,NaN,NaN,not provided,NaN,NaN,0


## TT split

In [11]:
def tt_split(df, select_floats=False):
    df = df.copy()
    if select_floats:
        df = df.select_dtypes(include=['int64', 'float64']).apply(lambda c: c.astype(float))
    df.fillna(-999, inplace=True)
    y = df.y
    del df['y']
    X = df
    cat_featues = np.where(X.dtypes != np.float)[0]
    X = X.apply(lambda c: c.fillna('NaN') if (c.dtype == object) else c)
    return train_test_split(X, y, test_size=0.2), cat_featues

In [12]:
(X_train, X_test, y_train, y_test), cat_features = tt_split(df)
len(X_train), len(X_test)

(189366, 47342)

## Catboost

In [13]:
model = CatBoostClassifier(iterations=300)

In [14]:
model.fit(X_train, y_train, cat_features=cat_features);

0:	learn: 0.6470503	total: 619ms	remaining: 3m 4s
1:	learn: 0.6048741	total: 1.05s	remaining: 2m 36s
2:	learn: 0.5675434	total: 1.31s	remaining: 2m 9s
3:	learn: 0.5354427	total: 1.55s	remaining: 1m 54s
4:	learn: 0.5037780	total: 1.83s	remaining: 1m 48s
5:	learn: 0.4776800	total: 2.26s	remaining: 1m 50s
6:	learn: 0.4541107	total: 2.52s	remaining: 1m 45s
7:	learn: 0.4320718	total: 2.91s	remaining: 1m 46s
8:	learn: 0.4126516	total: 3.31s	remaining: 1m 47s
9:	learn: 0.3921014	total: 3.64s	remaining: 1m 45s
10:	learn: 0.3758714	total: 4.01s	remaining: 1m 45s
11:	learn: 0.3596446	total: 4.43s	remaining: 1m 46s
12:	learn: 0.3456241	total: 4.73s	remaining: 1m 44s
13:	learn: 0.3322116	total: 5.01s	remaining: 1m 42s
14:	learn: 0.3169630	total: 5.49s	remaining: 1m 44s
15:	learn: 0.3067097	total: 5.8s	remaining: 1m 42s
16:	learn: 0.2967194	total: 6.22s	remaining: 1m 43s
17:	learn: 0.2879017	total: 6.49s	remaining: 1m 41s
18:	learn: 0.2785682	total: 6.96s	remaining: 1m 43s
19:	learn: 0.2697334	tota

157:	learn: 0.1135256	total: 1m 35s	remaining: 1m 25s
158:	learn: 0.1133363	total: 1m 36s	remaining: 1m 25s
159:	learn: 0.1132580	total: 1m 37s	remaining: 1m 25s
160:	learn: 0.1131490	total: 1m 38s	remaining: 1m 24s
161:	learn: 0.1130838	total: 1m 38s	remaining: 1m 24s
162:	learn: 0.1129611	total: 1m 39s	remaining: 1m 23s
163:	learn: 0.1129090	total: 1m 40s	remaining: 1m 23s
164:	learn: 0.1127454	total: 1m 41s	remaining: 1m 22s
165:	learn: 0.1126602	total: 1m 41s	remaining: 1m 22s
166:	learn: 0.1125689	total: 1m 42s	remaining: 1m 21s
167:	learn: 0.1124962	total: 1m 43s	remaining: 1m 21s
168:	learn: 0.1124654	total: 1m 44s	remaining: 1m 20s
169:	learn: 0.1124081	total: 1m 45s	remaining: 1m 20s
170:	learn: 0.1123092	total: 1m 45s	remaining: 1m 19s
171:	learn: 0.1122682	total: 1m 46s	remaining: 1m 19s
172:	learn: 0.1121866	total: 1m 47s	remaining: 1m 18s
173:	learn: 0.1121392	total: 1m 47s	remaining: 1m 17s
174:	learn: 0.1120560	total: 1m 48s	remaining: 1m 17s
175:	learn: 0.1119473	total:

In [15]:
y_pred = model.predict(X_test)
(y_pred == y_test).sum() / len(y_test)

0.9638587300916733

In [20]:
model.get_feature_importance(X_train, y_train, cat_features=cat_features)

[0.21311777825280845,
 5.318926816268187,
 1.4715188210104928,
 1.8375971503354274,
 7.075288206272252,
 12.235761454577075,
 6.553164179714665,
 0.23089090684192132,
 0.06870046908006486,
 0.027223380274787694,
 0.19494600203300924,
 0.028709163222891413,
 0.0,
 16.9426789876947,
 3.4000906920544978,
 0.36041670916499446,
 10.368962130018621,
 2.5695291162796585,
 0.15668271106822385,
 24.16024711709326,
 2.794824248343902,
 3.9907239603985576]